# SteganoGAN in Keras
This notebook contains code attempting to reimplement SteganoGAN in Keras, for the purpose of better understanding (and scrutinizing) it.

*Based on https://github.com/DAI-Lab/SteganoGAN/tree/master/steganogan*

### Modules

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import tensorflow as tf
import numpy as np
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from keras.utils import plot_model

from models import steganogan_encoder_dense_model, steganogan_decoder_dense_model, steganogan_critic_model
from keras_steganogan import KerasSteganoGAN

### Constants

In [18]:
# Image dimensions
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128
IMAGE_CHANNELS = 3

IMAGE_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)
MESSAGE_DEPTH = 2
BATCH_SIZE = 4
MODEL_PATH = 'pretrained_models/steganoGAN_similarity.keras'

### Call main encode and decode functions (with creating steganographic image and decoding it)

In [28]:
steganoGAN = KerasSteganoGAN(image_shape=IMAGE_SHAPE, data_depth=MESSAGE_DEPTH, model_path=MODEL_PATH)
steganoGAN.encode('input_1.png', 'images/output_similarity_1.png', 'Hello, World!')
steganoGAN.decode('images/output_similarity.png')

'Hç½}nøÀ{h¾±\x8cr\x9dª\xadêªªZ\x00\x11Öñ4A\x17ù¶k\x11\x842?\x04\x10Pri}J¿\x16çÓ\x00P\x02@\x00\x02ªÈ*h¾D\x00\x00\x00H\x97tnË£@\x00\ns\x1b\x06(³5\x9b²_¿ÕÛ÷u[Ä\x1e}÷\xa0**\xa0\x82Ü:¢\x01°äº\x0b@Ð\x98§8I6~;=&2Ä\x945äð\\\x13\x92ì¨¢bvNbDä\x19ú\x86\x06U.\x83\x1eò7i³ôÂµO~Îâî¤ú®²\x12\x8b)\x88vb¡òlôÎ\x89{þ\x19\xa0K7î¹\r½#\x83ÜÙ´C^p!´ð\x07B\x9cK`_ÐúÃ;\x02ÓQç\x18ð³Æ>]\x86÷\xadÌª\x1cx&ã±ñx¦\x0c\x91Å`y\x8f8\x83gº\x19æiõ!³Í¼\x137\x9a\x87ýÂ:t\x1azîà6f)òÃc\x1ajÃ\x88\x1f!\x13f`t\x7f³N)©Òñ\x0fÙ\x8bjä\x06ú5w°\x18¡éø|\x87±\x16Ë\x1e\x00xù¼ð\x1bóV6½Sf¦îå\x96ÏðôLe\xa0\x80kWçð·\x1eK\x80Ç\x88_Ìb$\x8d\x16M+\x93±\x9a\x86í¶mQ4{ªè(îã3\x9fÀ÷:\x10ÉÏ\x8c\x1f\x04\x9f :ù³ÍR¯sòÙ\x8aøÓ²x\x00\x00\x02\x82k¿i zÓ»\x1eq£\x88ª\x03\x9f4\x04$<½\x16±\x9cI¤\x11ü\x93`¯û¢\x0b\x03ëá\x83£\x89ÖÝ==\x7flüg>îªÉ\rR1\x0eê×\n\x18Bù¿ÿùÿ\x95Âðþ\x06NÂn&]WN?=ÌmD¿ê¨\x9b»ûíáO4{WT|P¸M\x85ÌìÞ\x8b\x914,\xa0"\x0cÌÝÜ\x00.¿|~5\xad5c"ÿu\x00\x00\x10\x00"Ú\x07à\x88ô\x85à°°Ñ²³uÎØ\x00ß\x1b\x1eWúÃ\x1bý}ß[\x0eª\x8a\x08¯\x80\x7f\x00f\x92É3Ì²©³;\'qh\x8b@ppé,8K\x0

### SteganoGAN predict random data with metrics

In [26]:
cover_image = tf.random.uniform([1, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS], -1, 1, dtype=tf.float32)
message = tf.cast(tf.random.uniform([1, IMAGE_HEIGHT, IMAGE_WIDTH, MESSAGE_DEPTH], 0, 2, dtype=tf.int32), tf.float32)
stego_img, recovered_msg = steganoGAN.predict([cover_image, message])
print("BinaryCrossentropy: {0}".format(BinaryCrossentropy()(message, recovered_msg)))
print("PSNR: {0}".format(tf.reduce_mean(tf.image.psnr(cover_image, stego_img, 1))))
print("SSIM: {0}".format(tf.reduce_mean(tf.image.ssim(cover_image, stego_img, 1))))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
BinaryCrossentropy: 1.3282517194747925
PSNR: 17.170175552368164
SSIM: 0.6067938208580017


----

### Build model for future train

In [17]:
encoder = steganogan_encoder_dense_model(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS, MESSAGE_DEPTH)
decoder = steganogan_decoder_dense_model(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS, MESSAGE_DEPTH)
critic  = steganogan_critic_model(IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)

steganoGAN = KerasSteganoGAN(
  encoder=encoder,
  decoder=decoder,
  critic=critic,
  image_shape=IMAGE_SHAPE,
  data_depth=MESSAGE_DEPTH,
  model_path=MODEL_PATH
)

steganoGAN.compile(
  encoder_optimizer = Adam(learning_rate=1e-4, beta_1=0.5),
  decoder_optimizer = Adam(learning_rate=1e-4, beta_1=0.5),
  critic_optimizer = Adam(learning_rate=1e-4, beta_1=0.5),
  loss_fn = BinaryCrossentropy(from_logits=False)
)

#steganoGAN.models_summary()
#steganoGAN.summary()
#plot_model(steganoGAN.encoder, to_file='model_images/encoder.png', show_shapes=True, show_layer_names=True)
#plot_model(steganoGAN.decoder, to_file='model_images/decoder.png', show_shapes=True, show_layer_names=True)
#plot_model(steganoGAN.critic, to_file='model_images/critic.png', show_shapes=True, show_layer_names=True)

### Download div2k dataset and complete it with random message dataset of {0, 1}

In [4]:
train_dir = '/Users/dmitryhoma/Projects/phd_dissertation/state_2/SteganoGAN/research/data/div2k/train'
val_dir = '/Users/dmitryhoma/Projects/phd_dissertation/state_2/SteganoGAN/research/data/div2k/val'

train_image_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir, 
    label_mode=None, 
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    seed=123,
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    shuffle=True
)

val_image_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir, 
    label_mode=None, 
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    seed=123,
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    shuffle=True
)

def normalize_img(img):
    return (img / 127.5) - 1

train_image_ds = train_image_ds.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
val_image_ds = val_image_ds.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)

def create_message_tensor_for_training(batch_size, width, height, data_depth):
    message = tf.random.uniform([batch_size, width, height, data_depth], 0, 2, dtype=tf.int32)
    message = tf.cast(message, tf.float32)
    return message

def create_message_dataset(batch_size, num_batches, width, height, data_depth):
    message_tensors = [create_message_tensor_for_training(batch_size, width, height, data_depth) for _ in range(num_batches)]
    return tf.data.Dataset.from_tensor_slices(tf.concat(message_tensors, axis=0)).batch(batch_size)

train_message_ds = create_message_dataset(BATCH_SIZE, len(train_image_ds), IMAGE_HEIGHT, IMAGE_WIDTH, MESSAGE_DEPTH)
val_message_ds = create_message_dataset(BATCH_SIZE, len(val_image_ds), IMAGE_HEIGHT, IMAGE_WIDTH, MESSAGE_DEPTH)

train_ds = tf.data.Dataset.zip((train_image_ds, train_message_ds)).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = tf.data.Dataset.zip((val_image_ds, val_message_ds)).prefetch(buffer_size=tf.data.AUTOTUNE)

Found 800 files.
Found 100 files.


In [5]:
steganoGAN.build([(1, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS), (1, IMAGE_HEIGHT, IMAGE_WIDTH, MESSAGE_DEPTH)])
steganoGAN.fit(train_ds, epochs=10, validation_data=val_ds)
steganoGAN.save(MODEL_PATH)

Epoch 1/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 229s 1s/step - bpp: 2.0000 - critic_loss: -0.0027 - decoding_loss: 0.6424 - encoder_decoder_total_loss: 1.2077 - psnr: 32.6068 - realism_loss: 0.5080 - similarity_loss: 0.0573 - ssim: 0.9566 - val_bpp: 2.0000 - val_critic_loss: -6.5629e-04 - val_decoding_loss: 0.6419 - val_encoder_decoder_total_loss: 1.1764 - val_psnr: 34.9229 - val_realism_loss: 0.5001 - val_similarity_loss: 0.0344 - val_ssim: 0.9659
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 220s 1s/step - bpp: 2.0000 - critic_loss: -0.0016 - decoding_loss: 0.6275 - encoder_decoder_total_loss: 1.1706 - psnr: 34.9173 - realism_loss: 0.5104 - similarity_loss: 0.0327 - ssim: 0.9677 - val_bpp: 2.0000 - val_critic_loss: -3.6327e-04 - val_decoding_loss: 0.6327 - val_encoder_decoder_total_loss: 1.1617 - val_psnr: 36.3578 - val_realism_loss: 0.5053 - val_similarity_loss: 0.0237 - val_ssim: 0.9738
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 220s 1s/step - bpp: 2.0000 - critic_loss: -0.0015 - decoding_loss: 0.6

----

### Testing different functions 

In [ ]:
from tensorflow.keras.losses import BinaryCrossentropy

# Create two random tensors
tensor1 = tf.random.uniform((4, 128, 128, 2), minval=0, maxval=1, dtype=tf.int32)
tensor2 = tf.random.uniform((4, 128, 128, 2), minval=0, maxval=1, dtype=tf.float32)

# Calculate binary crossentropy
loss = BinaryCrossentropy(from_logits=False)
loss = loss(tf.constant([1, 1, 1]), tf.constant([1, 1, 1]))
#loss = loss(tensor1, tensor2)

loss.numpy().astype(np.uint8)

In [ ]:
first_value = next(iter(train_ds.take(1)))
print(first_value)